# State Management Exercise: Library Assistant

In this exercise, you'll build a library assistant that responds to borrowing, returning, and overdue book queries.  The state is treated as immutable—nodes should return dictionaries of updates rather than modifying the state directly.

## Part 1: Conceptual Questions

1. **Why is it important to treat the state as immutable in LangGraph?**

2. **What fields does `LibraryState` define, and how are they used in this exercise?**

3. **Why do the handlers return dictionaries of updates instead of modifying the state object?**

4. **What does the `next_step` function do in this workflow?**

## Part 2: Implement the Library Assistant

Follow the TODOs in the code cell below to complete the library assistant.  Be sure to:

- Add `messages` field to `LibraryState` that stores a `List[AnyMessage]` with an `add_messages` reducer.
- Add a `books_borrowed` field to `LibraryState` that stores a `List[str]` without a reducer to allow the ability to remove or add values.
- Add a `last_user_message` field to `LibraryState` to store the last human message received.
- Implement the handlers to return dictionaries containing `messages`, `books_borrowed` when appropriate, and `resolved`.
- Add an edge from `START` to the `router` node in the workflow.
- Connect each handler node to the `END` node.
- Compile the graph with an `InMemorySaver` to persist state across multiple invocations.
- Run the application to test your solution.

> NOTE:
>
> In your handler code, **DO NOT** mutate or modify the passed in `state`. Instead update the global state by returning a new dictionary with only the fields you want to update.  LangGraph will merge these updates into the existing state.

In [5]:
from langgraph.checkpoint.memory import InMemorySaver
from typing import List, Optional, Literal, Dict, Any, Annotated, TypedDict
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, AnyMessage
from langgraph.graph.message import add_messages


class LibraryState(TypedDict):
    # TODO: Add a messages field that stores a List[AnyMessage] with an add_messages reducer
    section: Optional[Literal['borrow', 'return', 'overdue', 'unknown']]
    messages: Annotated[List[AnyMessage], add_messages]
    # TODO: Add a books_borrowed field that stores a List[str] without a reducer
    books_borrowed: List[str]
    resolved: bool
    #TODO: Add a last_user_message field to store the last human message received
    last_user_message: HumanMessage

# Router: return a partial update dict
def route_library(state: LibraryState) -> Dict[str, Any]:
    # Find the last human message
    last_msg = ''
    for msg in reversed(state.get('messages', [])):
        if isinstance(msg, HumanMessage):
            last_msg = msg.content.lower()
            break

    if 'borrow' in last_msg:
        intent = 'borrow'
    elif 'return' in last_msg:
        intent = 'return'
    elif 'overdue' in last_msg or 'fine' in last_msg:
        intent = 'overdue'
    else:
        intent = 'unknown'

    # Return state updates
    return {
        'last_user_message': last_msg,
        'section': intent,
        'resolved': False
    }

# Borrow handler
def handle_borrow(state: LibraryState) -> Dict[str, Any]:
    # TODO: extract the title after 'borrow'
    # Build and return a new dict with updates:
    # - 'books_borrowed': a new list with the new title appended if not already borrowed
    # - 'messages': a list containing one AIMessage response
    # - 'resolved': True

    book_title = None
    for msg in reversed(state.get('messages', [])):
        if isinstance(msg, HumanMessage):
            content = msg.content
            if 'borrow' in content.lower():
                # Extract book title after 'borrow'
                book_title = content.split('borrow', 1)[1].strip().title()
                break

    current_books = state.get('books_borrowed', [])
    updates = {}

    if book_title:
        if book_title not in current_books:
            # Replace the entire list with updated version
            updates['books_borrowed'] = current_books + [book_title]
            ai_text = f"Sure! I've added '{book_title}' to your borrowed books."
        else:
            ai_text = f"It looks like '{book_title}' is already on your list."
    else:
        ai_text = "I'm sorry, I couldn't identify the book you'd like to borrow."

    return {
        'messages': [AIMessage(content=ai_text)],
        **updates,
        'resolved': True
    }

# Return handler
def handle_return(state: LibraryState) -> Dict[str, Any]:
    # TODO: extract the title from the user's message after the word: 'return'
    # Build and return a dict:
    # - If the title is in books_borrowed, provide a new 'books_borrowed' list without it
    # - Always include 'messages' with an AIMessage response and set 'resolved': True
    book_title = None
    for msg in reversed(state.get('messages', [])):
        if isinstance(msg, HumanMessage):
            content = msg.content
            if 'return' in content.lower():
                # Extract book title after 'borrow'
                book_title = content.split('return', 1)[1].strip().title()
                break

    current_books = state.get('books_borrowed', [])
    updates = {}

    if book_title and book_title in current_books:
        # Create a new list without the returned book (no direct mutation)
        updated_books = [book for book in current_books if book != book_title]
        ai_text = f"Thank you! I've removed '{book_title}' from your borrowed books."

        return {
                'messages': [AIMessage(content=ai_text)],
                'books_borrowed': updated_books,
                'resolved': True
                }
    else:
         ai_text = f"Thank you! You have not '{book_title}' in your borrowed books."
         return {
                'messages': [AIMessage(content=ai_text)],
                'books_borrowed': current_books,
                'resolved': True
                }

# Overdue handler
def handle_overdue(state: LibraryState) -> Dict[str, Any]:
    current_books = state.get('books_borrowed', [])

    if current_books:
        books_list = ', '.join([f"'{book}'" for book in current_books])
        ai_text = f"You currently have {len(current_books)} book(s) borrowed: {books_list}. Please visit the library to check if any fines apply."
    else:
        ai_text = "You have no books borrowed at the moment. There are no overdue fines."

    return {
        'messages': [AIMessage(content=ai_text)],
        'resolved': True
        }

# Unknown handler: return clarification request
def handle_unknown(state: LibraryState) -> Dict[str, Any]:
    return {
        'messages': [AIMessage(content="I'm sorry, I didn't understand your request. Could you please clarify? I can help you borrow books, return books, or check for overdue items.")],
        'resolved': True
    }

# Decide the next step based on whether the request has been resolved
def next_step(state: LibraryState) -> str:
    if state.get("resolved", False):
        return END
    section = state.get("section", None)
    return section if section else END

# Build workflow
workflow = StateGraph(LibraryState)
workflow.add_node('router', route_library)
workflow.add_node('borrow', handle_borrow)
workflow.add_node('return', handle_return)
workflow.add_node('overdue', handle_overdue)
workflow.add_node('unknown', handle_unknown)

# TODO Add edge to connect START to 'router'
#TODO: Connect each handler to the END node

workflow.set_entry_point("router")
# Conditional edges from router to handlers
workflow.add_edge(START, "router")
workflow.add_conditional_edges('router', next_step, {
    'borrow': 'borrow',
    'return': 'return',
    'overdue': 'overdue',
    'unknown': 'unknown',
    END: END
})

workflow.add_edge("borrow", END)
workflow.add_edge("return", END)
workflow.add_edge("overdue", END)
workflow.add_edge("unknown", END)
memory = InMemorySaver()


if __name__ == "__main__":
    # TODO: compile with InMemorySaver and test multiple invocations with same thread_id
    app = workflow.compile(checkpointer=memory)

    config = {"configurable": {"thread_id": 'demo_user'}}

    # First interaction: borrow a book
    state1 = {
        'messages': [HumanMessage(content="I want to borrow Moby Dick")],
        'books_borrowed': [],
        'resolved': False
    }

    result1 = app.invoke(state1, config=config)
    print("Result 1 - Borrow:")
    print(f"  User message: {result1.get('last_user_message', '')}")
    print(f"  Books borrowed: {result1.get('books_borrowed', [])}")
    print(f"  Last message: {result1['messages'][-1].content}\n")

    # Second interaction: check overdue (properly create new state)
    current_state = app.get_state(config).values
    state2 = {
        **current_state,
        'messages': current_state['messages'] + [HumanMessage(content="Are there any overdue books?")]
    }
    result2 = app.invoke(state2, config=config)
    print("Result 2 - Check overdue:")
    print(f"  User message: {result2.get('last_user_message', '')}")
    print(f"  Books borrowed: {result2.get('books_borrowed', [])}")
    print(f"  Last message: {result2['messages'][-1].content}\n")

    # Third interaction: return the book (properly create new state)
    current_state = app.get_state(config).values
    state3 = {
        **current_state,
        'messages': current_state['messages'] + [HumanMessage(content="I need to return Moby Dick")]
    }

    result3 = app.invoke(state3, config=config)
    print("Result 3 - Return:")
    print(f"  User message: {result3.get('last_user_message', '')}")
    print(f"  Books borrowed: {result3.get('books_borrowed', [])}")
    print(f"  Last message: {result3['messages'][-1].content}\n")

    # Fourth interaction: check status again
    current_state = app.get_state(config).values
    state4 = {
        **current_state,
        'messages': current_state['messages'] + [HumanMessage(content="Do I have any overdue books?")]
    }
    result4 = app.invoke(state4, config=config)
    print("Result 4 - Final check:")
    print(f"  User message: {result4.get('last_user_message', '')}")
    print(f"  Books borrowed: {result4.get('books_borrowed', [])}")
    print(f"  Last message: {result4['messages'][-1].content}")

Result 1 - Borrow:
  User message: i want to borrow moby dick
  Books borrowed: ['Moby Dick']
  Last message: Sure! I've added 'Moby Dick' to your borrowed books.

Result 2 - Check overdue:
  User message: are there any overdue books?
  Books borrowed: ['Moby Dick']
  Last message: You currently have 1 book(s) borrowed: 'Moby Dick'. Please visit the library to check if any fines apply.

Result 3 - Return:
  User message: i need to return moby dick
  Books borrowed: []
  Last message: Thank you! I've removed 'Moby Dick' from your borrowed books.

Result 4 - Final check:
  User message: do i have any overdue books?
  Books borrowed: []
  Last message: You have no books borrowed at the moment. There are no overdue fines.
